<a href="https://colab.research.google.com/github/LinYenShou/114-1/blob/main/%E5%87%BA%E9%81%8A%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-search-results folium geopy -q

  Preparing metadata (setup.py) ... done


In [2]:
from serpapi import GoogleSearch
import folium
from geopy.geocoders import Nominatim
import json
import re
from datetime import datetime
from IPython.display import display

In [3]:
SERPAPI_KEY = "fb13aee3bf83990b933394082ac6e5673fadae6d8f5cbf9e45d7fd57e990781f"

In [4]:
# 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")


TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei',
    '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan',
    '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan',
    '高雄', '高雄市', 'Kaohsiung',

    # 縣市
    '基隆', '基隆市', 'Keelung',
    '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli',
    '彰化', '彰化縣', 'Changhua',
    '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin',
    '嘉義', '嘉義市', '嘉義縣', 'Chiayi',
    '屏東', '屏東縣', 'Pingtung',
    '宜蘭', '宜蘭縣', 'Yilan',
    '花蓮', '花蓮縣', 'Hualien',
    '台東', '台東縣', '臺東', '臺東縣', 'Taitung',
    '澎湖', '澎湖縣', 'Penghu',
    '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',

    # 知名景點/地標
    '101', '台北101', '西門町', '信義區', '中正區',
    '士林', '北投', '淡水', '九份', '平溪',
    '日月潭', '阿里山', '墾丁', '太魯閣',
    '東大門', '逢甲', '一中街', '勤美',
]

def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    """
    使用 SerpAPI 進行 Google 搜尋
    """
    try:
        params = {
            'q': query,
            'api_key': api_key,
            'num': num,
            'gl': gl,
            'hl': hl
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        return results.get('organic_results', [])

    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
        return []


def extract_locations_from_text(text):
    """
    從文字中擷取地區名稱

    參數:
        text (str): 要分析的文字

    回傳:
        list: 找到的地區名稱列表
    """
    found_locations = []
    text_lower = text.lower()

    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            # 避免重複加入相似地名
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)

    return found_locations


def get_coordinates(location_name):
    """
    取得地點的經緯度座標

    參數:
        location_name (str): 地點名稱

    回傳:
        tuple: (緯度, 經度) 或 None
    """
    try:
        # 加上台灣以提高準確度
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print(f"無法取得 {location_name} 的座標: {e}")
        return None


def analyze_results_with_locations(results):
    """
    分析搜尋結果並擷取地理位置資訊

    參數:
        results (list): 搜尋結果列表

    回傳:
        list: 包含位置資訊的結果字典列表
    """
    analyzed_results = []

    for result in results:
        # 從標題和描述中尋找地點
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"

        locations = extract_locations_from_text(combined_text)

        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })

    return analyzed_results


def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    """
    根據分析結果建立地圖

    參數:
        analyzed_results (list): 包含位置資訊的結果列表
        center (list): 地圖中心座標 [緯度, 經度]
        zoom (int): 地圖縮放層級

    回傳:
        folium.Map: 地圖物件
    """
    # 建立地圖
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')

    # 用於記錄已標註的位置，避免重複
    marked_locations = {}

    # 為每個結果加上標記
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)

            if coords:
                lat, lon = coords

                # 如果該位置還沒標記過
                if location not in marked_locations:
                    marked_locations[location] = []

                # 將結果加到該位置
                marked_locations[location].append(result)

    # 在地圖上標註
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords

            # 建立 popup 內容
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"

            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"

            popup_html += "</div>"

            # 加上標記
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m


def display_results_summary(analyzed_results):
    """
    顯示分析結果摘要
    """
    print("\n" + "="*80)
    print(f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果")
    print("="*80 + "\n")

    for i, result in enumerate(analyzed_results, 1):
        print(f"【結果 {i}】")
        print(f"標題: {result['title']}")
        print(f"地點: {', '.join(result['locations'])}")
        print(f"描述: {result['snippet'][:100]}...")
        print(f"網址: {result['link']}")
        print("-"*80 + "\n")

In [5]:
def search_and_map(query, api_key, num_results=10):
    """
    整合搜尋與地圖標註的主函數

    參數:
        query (str): 搜尋關鍵字
        api_key (str): SerpAPI 金鑰
        num_results (int): 搜尋結果數量

    回傳:
        folium.Map: 標註好的地圖
    """
    print(f"正在搜尋: {query}\n")

    # 1. 執行搜尋
    results = search_google(query, api_key, num=num_results)

    if not results:
        print("沒有找到任何結果")
        return None

    print(f"✓ 找到 {len(results)} 筆搜尋結果")

    # 2. 分析結果並擷取地理位置
    analyzed_results = analyze_results_with_locations(results)

    if not analyzed_results:
        print("\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區")
        return None

    print(f"✓ 從結果中找到 {len(analyzed_results)} 筆包含地理位置的資料\n")

    # 3. 顯示結果摘要
    display_results_summary(analyzed_results)

    # 4. 建立地圖
    print("正在建立地圖...\n")
    map_obj = create_map_from_results(analyzed_results)
    print("✓ 地圖建立完成！")

    return map_obj

In [7]:
#搜尋台灣美食
query = "台灣哪裡好玩推薦"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣哪裡好玩推薦

✓ 找到 8 筆搜尋結果
✓ 從結果中找到 7 筆包含地理位置的資料


找到 7 筆包含地理位置的搜尋結果

【結果 1】
標題: 2025台灣景點一日遊懶人包，國內旅遊
地點: 桃園
描述: 2025推薦超過60個「桃園景點」推薦懶人包，從觀光工廠玩到森林瀑布一日遊，桃園市除了到拉拉山玩之外，千萬千萬不要忘了安排桃園景點一日遊，直接整理推薦超過 ......
網址: https://bunnyann.tw/attractions/
--------------------------------------------------------------------------------

【結果 2】
標題: 2025 年台灣絕美秘境：嚴選40 個從北到南的必去推薦景點
地點: 台北
描述: 台灣北部必去秘境推薦 · 推薦【台北必去秘境】圓山大飯店密道 · 推薦【台北內湖必去秘境】大溝溪生態治水園區 · 推薦【台北公館必去秘境】台電加羅林魚木....
網址: https://hshsharehouse.com/taiwan-hidden-gems-40-must-visit-destinations-north-to-south/
--------------------------------------------------------------------------------

【結果 3】
標題: 觀光景點
地點: 苗栗, 雲林
描述: 四季氣溫宜人，擁有多處老少皆宜的渡假村及遊樂中心；喜歡文化和文藝則不可錯過眾多美術館及博物館、苗栗木雕與燒陶藝術，以及雲林布袋戲演藝文化，值得好好探訪和體驗。...
網址: https://www.taiwan.net.tw/m1.aspx?sNo=0001016
--------------------------------------------------------------------------------

【結果 4】
標題: 【2025台灣自由行】新手必看《 32 個台灣行程&住宿》(含景點 ...
地點: 台中, 苗栗, 南投
描述: 很多爆紅的台灣景點都在中台灣，台中的歌劇院、高美濕地、路思義教堂；苗栗的勝興車站、龍騰斷橋；南投的天空之橋、忘憂森林等等